# Kapitel 10: Textanalyse historischer Daten mit NLTK (und optional spaCy/CLTK)

Dieses Notebook ist als **instruktive Begleitung für Kapitel 10** gedacht und arbeitet mit **echten Korpora aus `nltk.corpus`** und wenigen, aber zusammenhängenden historischen Beispieltexten.

## Lernziele (Notebook)
- Korpusdaten mit `nltk.corpus` laden und inspizieren
- Tokenisierung, Type–Token-Relation und Stoppwörter praktisch nachvollziehen
- Wortfrequenzen und Collocations (Kollokationen) untersuchen
- Ein einfaches diachrones Beispiel mit den US-Inauguralreden durchführen
- Optional: Named Entity Recognition (NER) mit spaCy und ein kurzer Ausblick auf CLTK für Latein


## 1. Setup

In dieser Zelle werden die benötigten Bibliotheken geladen und die wichtigsten NLTK-Korpora nachgeladen.

Für die Videos kannst du hier kurz erklären:
- Was `nltk` ist
- Was ein "Korpus" ist
- Warum wir ausgewählte Korpora (Genesis, Gutenberg, Inaugural) verwenden.

In [ ]:
import nltk
import matplotlib.pyplot as plt

from nltk.corpus import gutenberg, genesis, inaugural
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk import Text, bigrams

# NLTK-Resources sicherheitshalber nachladen (idempotent)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('gutenberg')
nltk.download('genesis')
nltk.download('inaugural')

print("NLTK-Version:", nltk.__version__)

## 2. Überblick: Verwendete Korpora

Hier verschaffst du dir (und den Studierenden) einen Überblick über einige historische Korpora, die bereits in NLTK enthalten sind:

- **Genesis** (englische King-James-Version) – biblischer Text, frühneuzeitliches Englisch
- **Gutenberg** – literarische Klassiker (z. B. Shakespeare, Austen, Melville)
- **Inaugural** – US-Präsidenten-Antrittsreden (politischer Diskurs von 1789 bis in die Gegenwart)

In [ ]:
print("Genesis fileids:", genesis.fileids())
print("\nGutenberg fileids:")
print(gutenberg.fileids())
print("\nInaugural fileids (erste 10):")
print(inaugural.fileids()[:10])

## 3. Fallstudie 1: Genesis (KJV) – Tokenisierung & Type–Token-Relation

Wir nutzen das Korpus `genesis` (King James Version) als Beispiel für einen **kohärenten historischen Text**.

Schritte:
1. Rohtext ansehen (Ausschnitt)
2. Tokenisierung
3. Zählen von Tokens und Types
4. Type–Token-Relation (TTR) berechnen


In [ ]:
# Rohtext aus dem Genesis-Korpus (englische KJV)
text_genesis = genesis.raw('english-kjv.txt')

print("Ausschnitt (erste 500 Zeichen):\n")
print(text_genesis[:500])

In [ ]:
from nltk.tokenize import word_tokenize

# Tokenisierung
tokens_genesis = word_tokenize(text_genesis)

n_tokens = len(tokens_genesis)
n_types = len(set(w.lower() for w in tokens_genesis))
ttr = n_types / n_tokens

print("Anzahl Tokens:", n_tokens)
print("Anzahl Types:", n_types)
print("Type–Token-Relation (TTR):", round(ttr, 4))

print("\nDie ersten 30 Tokens:")
print(tokens_genesis[:30])

## 4. Stoppwörter & Wortfrequenzanalyse

Nun betrachten wir die häufigsten Wörter in Genesis.

Schritte:
1. Frequenzverteilung **mit** Stoppwörtern
2. Frequenzverteilung **ohne** Stoppwörter
3. Plot der häufigsten Wörter (log-Skala)

In [ ]:
# Frequenzen MIT Stoppwörtern
fd_all = FreqDist(w.lower() for w in tokens_genesis)
print("Häufigste 20 Tokens (mit Stoppwörtern):")
print(fd_all.most_common(20))

In [ ]:
# Frequenzen OHNE Stoppwörter
en_stop = set(stopwords.words('english'))
tokens_genesis_nostop = [w.lower() for w in tokens_genesis if w.isalpha() and w.lower() not in en_stop]
fd_nostop = FreqDist(tokens_genesis_nostop)

print("Häufigste 20 Tokens (ohne Stoppwörter):")
print(fd_nostop.most_common(20))

In [ ]:
# Plot der 30 häufigsten Wörter (ohne Stoppwörter) in log-Skala
plt.figure(figsize=(10,4))
fd_nostop.plot(30, cumulative=False)
plt.yscale('log')
plt.title('Häufigste Wörter in Genesis (ohne Stoppwörter, log-Skala)')
plt.tight_layout()
plt.show()

## 5. Kollokationen in Genesis

Als nächstes suchen wir nach **Kollokationen**, also Wörtern, die häufig zusammen auftreten. Das ist besonders spannend für historische Fragestellungen:
- Welche Begriffe treten gemeinsam mit *God* auf?
- Welche typischen Formulierungen verwendet ein Text?

Wir verwenden dafür `nltk.Text` und die eingebaute Methode `.collocations()`.

In [ ]:
text_obj_genesis = Text(tokens_genesis)

print("Kollokationen in Genesis (Top 20):")
text_obj_genesis.collocations(num=20)

## 6. Fallstudie 2: Politischer Diskurs – Inauguralreden & das Wort „freedom“

Nun wechseln wir zu einem explizit politischen Korpus: den **US-Präsidenten-Antrittsreden** (`inaugural`).

Wir betrachten als einfache diachrone Fragestellung:
- Wie häufig kommt das Wort **„freedom“** in den verschiedenen Reden vor (relativ zur Länge der Rede)?


In [ ]:
fileids = inaugural.fileids()

data = []  # (jahr, president, rel_freq)

for fid in fileids:
    # Dateiname: 'YYYY-PresName.txt'
    year_str, pres = fid.split('-')[0], '-'.join(fid.split('-')[1:]).replace('.txt', '')
    year = int(year_str)
    words = [w.lower() for w in inaugural.words(fid)]
    total = len(words)
    freq = words.count('freedom') / total if total > 0 else 0
    data.append((year, pres, freq))

data_sorted = sorted(data, key=lambda x: x[0])

for year, pres, freq in data_sorted[:10]:
    print(f"{year}: {pres:20s} -> rel. freq 'freedom': {freq:.4f}")

In [ ]:
# Plot der relativen Häufigkeit von 'freedom' über die Zeit
years = [d[0] for d in data_sorted]
freqs = [d[2] for d in data_sorted]

plt.figure(figsize=(10,4))
plt.plot(years, freqs, marker='o')
plt.xlabel('Jahr der Inauguralrede')
plt.ylabel("Relative Häufigkeit von 'freedom'")
plt.title("'freedom' in US-Inauguralreden")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Optional: Named Entity Recognition (NER) mit spaCy

Für eine **erste Demonstration von NER** kannst du `spaCy` auf eine einzelne Inauguralrede anwenden.

Wir nehmen z. B. die erste Rede von Barack Obama (2009). Wir können anhand der Eigennamenerkennung:
- zeigen, welche Personen, Orte, Organisationen im Text vorkommen
- den Link schlagen zur historischen Netzwerkanalyse, Ortsregistern etc.

Wenn `spaCy` oder das Modell nicht installiert ist, überspringt die Zelle den Code.

In [ ]:
try:
    import spacy
    nlp_en = spacy.load("en_core_web_sm")
    spacy_available = True
except Exception as e:
    print("spaCy oder das englische Modell sind nicht verfügbar:", e)
    spacy_available = False

if spacy_available:
    obama_text = inaugural.raw('2009-Obama.txt')
    print("Ausschnitt aus der Inauguralrede 2009 (erste 500 Zeichen):\n")
    print(obama_text[:500])

    doc = nlp_en(obama_text)
    print("\nGefundene Entitäten (erste 30):")
    for ent in list(doc.ents)[:30]:
        print(ent.text, "->", ent.label_)
else:
    print("NER-Beispiel mit spaCy wird übersprungen.")

## 8. Optional: Kurzer Ausblick – Latein mit CLTK

Zum Schluss ein **kurzer Ausblick auf CLTK** mit einem zusammenhängenden lateinischen Text.

Wir verwenden einen kurzen Auszug aus Caesars *Bellum Gallicum* (1,1):

> *Gallia est omnis divisa in partes tres, quarum unam incolunt Belgae, aliam Aquitani, tertiam qui ipsorum lingua Celtae, nostra Galli appellantur.*

Wir zeigen nur: Tokenisierung & Lemmatisierung.
Wenn CLTK nicht installiert ist, wird das Beispiel übersprungen.

In [ ]:
try:
    from cltk import NLP as CLTK_NLP
    cltk_available = True
except Exception as e:
    print("CLTK ist nicht verfügbar:", e)
    cltk_available = False

latin_caesar = (
    "Gallia est omnis divisa in partes tres, quarum unam incolunt Belgae, "
    "aliam Aquitani, tertiam qui ipsorum lingua Celtae, nostra Galli appellantur."
)

print(latin_caesar)

if cltk_available:
    nlp_lat = CLTK_NLP("lat")
    doc_lat = nlp_lat(latin_caesar)

    print("\nTokens:")
    print(doc_lat.tokens)

    print("\nLemmata:")
    print(doc_lat.lemmata)
else:
    print("Latein-Beispiel mit CLTK wird übersprungen.")

---

## 9. Zusammenfassung & Transfer

In diesem Notebook hast du anhand **kohärenter historischer Texte** gesehen, wie man:

- Korpora aus `nltk.corpus` lädt und inspiziert
- Tokenisierung, Type–Token-Relation und Stoppwörter praktisch nutzt
- Wortfrequenzen und Kollokationen in einem biblischen Text (Genesis) analysiert
- eine einfache diachrone Untersuchung in politischen Reden (Inauguralreden) durchführt
- optional NER mit spaCy und lateinische Analysen mit CLTK andeutet.